In [1]:
import numpy as np
import pandas as pd
import sys, os
from collections import defaultdict
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

sys.path.append('../Shared')
import processing, Model, Visualization


def call_data(base_path, sub_lst, sub_idx):
    data = pd.read_csv(base_path+sub_lst[sub_idx])
    return data

def data_setup(lst_normal, path_normal, window_size, step_size):
    subjects_data = defaultdict(list)
    subjects_label = defaultdict(list)

    for s_idx, filename in enumerate(lst_normal):
        name = filename.lower()
        subject_id = name.split("n")[0]
        data = call_data(path_normal, lst_normal, s_idx).to_numpy()  # (num_samples, num_channels)

        windows = processing.sliding_window(data, window_size, step_size)  # (num_windows, win_len, ch)

        if "standing" in name: label = 0
        elif "gait" in name: label = 1
        elif "sitting" in name: label = 2
        else: label = -1

        labels = np.full(len(windows), label)

        subjects_data[subject_id].append(windows)
        subjects_label[subject_id].append(labels)

    return subjects_data, subjects_label



def get_X_y(X, y):
    all_X, all_y = [], []

    #for subject_id in subjects_data.keys():
    data_list = X
    label_list = y

    for data, labels in zip(data_list, label_list):
        for w, label in zip(data, labels):  # w: (win_len, ch)
            feat = processing.extract_features(w)  # (num_channels*5,)
            all_X.append(feat)
            all_y.append(label)

    all_X, all_y = np.array(all_X), np.array(all_y)   # (N, num_channels*5)

    return all_X, all_y

In [ ]:
import importlib
importlib.reload(processing)

In [8]:
path_abnormal = "../../../Data/Gait1/Abnormal/"
path_normal = "../../../Data/Gait1/normal/"

lst_abnormal = os.listdir(path_abnormal)
lst_normal = os.listdir(path_normal)

window_size, step_size = 200, 10
num_channels, num_features = 5, 30  # we extracted 5 features per channel

subjects_data, subjects_label = data_setup(lst_normal, path_normal, window_size, step_size)

Features = ['mav', 'var', 'zc', 'iemg', 'wl', 'wamp', 'mavs', 'rms', 'ssc', 'msq', 'v3', 'ld', 'dabs', 'mfl', 'mpr', 'mnf', 'psr', 'arc1', 'arc2', 'arc3', 'arc4', 'cc1', 'cc2', 'cc3', 'cca', 'dwtc1', 'dwtc2', 'dwtpc1', 'dwtpc2', 'dwtpc3']

All_Acc = []
for subject_id in subjects_data.keys():
    print(f"Sub {subject_id} Start")
    X, y = get_X_y(X=subjects_data[subject_id], y=subjects_label[subject_id])
    X = X.reshape(-1, num_channels, num_features)

    Sub_acc = []
    for feature_idx in range(num_features):
        X_feature = X[:, :, feature_idx]   # shape: (samples, num_channels, 1, 1)
        X_feature = X_feature.reshape(-1, num_channels, 1)

        X_train, X_test, y_train, y_test = train_test_split(X_feature, y, test_size=0.2, random_state=42)

        model = Model.build_model_1D(input_shape=X_feature.shape[1:], num_classes=len(np.unique(y)))

        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=512, verbose=0)

        print(f"\tSub ID {subject_id}, Feature {feature_idx} ({Features[feature_idx]}): "
            f"val acc => {np.max(history.history['val_accuracy'])*100:.2f}%")
        Sub_acc.append(np.max(history.history['val_accuracy'])*100)
    All_Acc.append(Sub_acc)
    print("\n")

Sub 10 Start
	Sub ID 10, Feature 0 (mav): val acc => 80.67%
	Sub ID 10, Feature 1 (var): val acc => 60.29%
	Sub ID 10, Feature 2 (zc): val acc => 82.21%
	Sub ID 10, Feature 3 (iemg): val acc => 90.13%
	Sub ID 10, Feature 4 (wl): val acc => 98.04%
	Sub ID 10, Feature 5 (wamp): val acc => 77.80%
	Sub ID 10, Feature 6 (mavs): val acc => 61.13%
	Sub ID 10, Feature 7 (rms): val acc => 64.78%
	Sub ID 10, Feature 8 (ssc): val acc => 75.35%
	Sub ID 10, Feature 9 (msq): val acc => 60.29%
	Sub ID 10, Feature 10 (v3): val acc => 60.36%
	Sub ID 10, Feature 11 (ld): val acc => 66.74%
	Sub ID 10, Feature 12 (dabs): val acc => 92.51%
	Sub ID 10, Feature 13 (mfl): val acc => 53.50%
	Sub ID 10, Feature 14 (mpr): val acc => 72.48%
	Sub ID 10, Feature 15 (mnf): val acc => 74.51%
	Sub ID 10, Feature 16 (psr): val acc => 50.63%
	Sub ID 10, Feature 17 (arc1): val acc => 87.61%
	Sub ID 10, Feature 18 (arc2): val acc => 90.48%
	Sub ID 10, Feature 19 (arc3): val acc => 80.11%
	Sub ID 10, Feature 20 (arc4): val

In [9]:
pd.DataFrame(All_Acc.T)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,80.672270,60.294116,82.212883,90.126050,98.039216,77.801120,61.134452,64.775908,75.350142,60.294116,...,72.549021,50.630254,50.630254,50.630254,50.630254,60.644257,76.400560,60.574228,61.624652,50.630254
1,67.275494,65.449011,88.888890,65.753424,94.824964,77.016741,62.709284,78.843224,90.410960,64.535767,...,69.863015,47.336379,31.050229,31.050229,31.050229,67.427701,52.206999,68.949771,53.120244,47.336379
2,78.826898,58.941346,69.670957,83.977109,96.137339,73.247498,55.937052,79.685265,64.806867,53.075820,...,61.230332,41.058657,39.484978,39.484978,39.484978,59.084404,69.384837,69.241774,56.652361,47.925609
3,69.577080,55.252385,70.668483,58.253753,93.315142,73.806274,52.387446,69.577080,63.301504,63.574350,...,58.117324,38.472033,39.290586,39.290586,39.290586,68.349248,63.847202,66.984993,56.480217,40.654844
4,60.763359,65.801525,77.251911,68.702292,83.816797,72.366410,61.679387,65.343511,57.557249,60.152674,...,65.343511,48.702291,26.564884,26.564884,26.564884,68.549621,56.030536,66.717559,56.793892,48.702291
5,68.681318,51.373625,68.818682,76.923078,80.906594,69.093406,50.137365,73.489010,62.087911,53.983516,...,61.126375,43.131867,34.065935,34.065935,34.065935,55.219781,57.554942,54.258239,51.510990,48.763737
6,52.949643,55.539566,79.856116,69.496405,91.654676,75.683451,47.482014,59.424460,66.187048,52.374101,...,67.625898,42.014387,42.014387,42.014387,42.014387,48.345324,56.402880,46.187049,55.251801,42.014387
7,48.070174,56.257308,81.520468,79.649121,89.590645,74.385965,53.801167,56.725144,63.391811,54.502922,...,65.964913,48.070174,33.099416,33.099416,33.099416,54.853803,60.818714,55.204678,54.502922,48.070174
8,69.673914,61.304349,91.413045,72.500002,99.021739,74.021739,61.956519,77.934784,84.891301,63.804346,...,71.956521,48.043478,33.043477,33.043477,33.043477,75.760871,54.347825,81.086957,61.195654,58.913046
9,73.313785,62.365592,79.569894,56.598240,94.428152,78.690130,59.335285,74.877810,63.245356,66.177911,...,66.959924,44.183773,44.183773,35.874876,35.874876,71.554250,57.575756,71.163243,50.439882,44.183773


In [12]:
subs = ["S10", "S11", "S1", "S2","S3","S4","S5","S6","S7","S8","S9"]

pd.DataFrame(np.array(All_Acc).T, columns=subs).to_csv(f"./Feature_wise_acc.csv", index=False)